# Testing Regression

In [ ]:
# import processed df from ../data/processed/df_merged_extended_weather.csv
import pandas as pd
df_merged_extended_weather = pd.read_csv("../data/processed/df_merged_extended_weather.csv")
display(df_merged_extended_weather.shape)

In [ ]:


df_merged_extended_weather.tail()

#show only columns Datum,temperature_weather, temperature_2m_mean
#df_merged_extended_weather[['Datum', 'Temperatur_weather', 'temperature_2m_mean']].sample(10)

#calculate sample mean between temperature_weather and temperature_2m_mean
#df_merged_extended_weather['temperature_mean'] = df_merged_extended_weather[['Temperatur_weather', 'temperature_2m_mean']].mean(axis=1)
#df_merged_extended_weather['temperature_median'] = df_merged_extended_weather[['Temperatur_weather', 'temperature_2m_mean']].median(axis=1)
#df_merged_extended_weather[['Datum', 'Temperatur_weather', 'temperature_2m_mean','temperature_median','temperature_mean']].sample(10)

In [ ]:
import scipy.stats as stats
import seaborn as sns
# Testing Regression
#reg_cols=['Umsatz_umsatz','umsatz_rolling7','KielerWoche_kiwo','Bewoelkung_weather', 'rain_sum', 'sunshine_hours','Temperatur_weather','Windgeschwindigkeit_weather','precipitation_hours','day_of_week']
reg_cols=['Umsatz_umsatz','KielerWoche_kiwo','Bewoelkung_weather', 'rain_sum', 'sunshine_hours','Temperatur_weather','Windgeschwindigkeit_weather','precipitation_hours','day_of_week']
sns.pairplot(df_merged_extended_weather[reg_cols].dropna())
df_merged_extended_weather[reg_cols].corr()

In [ ]:
import statsmodels.formula.api as smf

# Example regression formula (adjust as needed)
formula = "Umsatz_umsatz ~  Temperatur_weather + Bewoelkung_weather + sunshine_hours  + C(Warengruppe_umsatz)"

# Drop rows with missing values in the regression columns
regression_df = df_merged_extended_weather.dropna(subset=[
    "Umsatz_umsatz", "Bewoelkung_weather", "rain_sum", "sunshine_hours","KielerWoche_kiwo",
    "Temperatur_weather", "Windgeschwindigkeit_weather", "precipitation_hours", "day_of_week","Warengruppe_umsatz"
])

# Fit the model
model = smf.ols(formula, data=regression_df).fit()

# Show summary
display(model.summary())

display(regression_df.head())



# Merging test data from test csv


In [ ]:
# Read test.csv into testdf
df_test = pd.read_csv("data/test.csv")

# Quick checks
print("Shape:", df_test.shape)
display(df_test.head())
display(df_test.tail())

In [ ]:
## add weather Temperatur_weather from df_weather
df_test['Datum'] = pd.to_datetime(df_test['Datum'], errors='coerce').dt.normalize()
df_weather['Datum'] = pd.to_datetime(df_weather['Datum'], errors='coerce').dt.normalize()


# Left-join temperature from df_weather onto the test DataFrame (keep all test rows)
df_test_weather = df_test.merge(
    df_weather[['Datum', 'Temperatur_weather', 'Bewoelkung_weather']],
    on='Datum',
    how='left'
)

# now adding day_of_week column
df_test_weather['day_of_week'] = df_test_weather['Datum'].dt.weekday

# filling sunshine_hours from meteo api
test_min_date = df_test_weather['Datum'].min()
test_max_date = df_test_weather['Datum'].max()
print (f"Fetching weather data from {test_min_date} to {test_max_date}...")
lat_kiel, lon_kiel = 54.3233, 10.1228
daily_vars = ['sunshine_duration']
df_sunshine_date = fetch_open_meteo_daily_range(test_min_date, test_max_date, daily_vars=daily_vars) #defaults to kiel lat/lon
df_sunshine_date['sunshine_hours'] = df_sunshine_date['sunshine_duration'] / 3600.0

# rename df_sunshine_date column date to Datum for merging
# drop sunshine_duration column
df_sunshine_date.drop(columns=['sunshine_duration'],inplace=True)
df_sunshine_date = df_sunshine_date.rename_axis('Datum').reset_index()

# convert Datum to type datetime
df_sunshine_date['Datum'] = pd.to_datetime(df_sunshine_date['Datum'], errors='coerce').dt.normalize()
display(df_sunshine_date.head())

# merge sunshine_hours into df_test_weather
df_test_weather = df_test_weather.merge(
    df_sunshine_date[['Datum', 'sunshine_hours']],
    on='Datum',
    how='left'
)

#print shape of df_test_weather
print("Shape after merging sunshine_hours:", df_test_weather.shape)

# merge kiwo data into df_test_weather
df_test_weather = df_test_weather.merge(
    df_kiwo[['Datum', 'KielerWoche_kiwo']],
    on='Datum',
    how='left'
)

print("Shape after merging kiwo data:", df_test_weather.shape)



# Quick sanity checks
print("Shape:", df_test_weather.shape)
print("Missing Temperatur_weather:", df_test_weather['Temperatur_weather'].isna().sum())
#check missing sunshine_hours
print("Missing sunshine_hours:", df_test_weather['sunshine_hours'].isna().sum())
#check missing Bewoelkung_weather
print("Missing Bewoelkung_weather:", df_test_weather['Bewoelkung_weather'].isna().sum())
#check missing KielerWoche_kiwo
print("Missing KielerWoche_kiwo:", df_test_weather['KielerWoche_kiwo'].isna().sum())

df_test_weather['KielerWoche_kiwo'] = df_test_weather['KielerWoche_kiwo'].fillna(0)

display(df_test_weather.head())
display(df_test_weather.tail())
# display 10 sample where kielerwoche_kiwo is 1
display(df_test_weather[df_test_weather['KielerWoche_kiwo'] == 1].sample(10))



In [ ]:
## interpolating missing Temperatur_weather
df_test_weather['Temperatur_weather'] = df_test_weather['Temperatur_weather'].interpolate()

## interpolate Bewoelkung_weather
df_test_weather['Bewoelkung_weather'] = df_test_weather['Bewoelkung_weather'].interpolate()

#15.0
#NaN
#17.0
# interpolation should fill the NaN with 16.0

print("Missing Temperatur_weather:", df_test_weather['Temperatur_weather'].isna().sum())
print("Missing Bewoelkung_weather:", df_test_weather['Bewoelkung_weather'].isna().sum())



## Predicting with df_test_weather

In [ ]:
#predicting with df_test_weather
# rename df_test_weather.Warengruppe to Warengruppe_umsatz for consistency
df_test_weather = df_test_weather.rename(columns={"Warengruppe": "Warengruppe_umsatz"})

# Required columns for this model
req_cols = ['Temperatur_weather', 'Warengruppe_umsatz']

# Sanity checks
missing = [c for c in req_cols if c not in df_test_weather.columns]
if missing:
    raise ValueError(f"Missing columns for prediction: {missing}")

# Ensure numeric temperature
df_test_weather['Temperatur_weather'] = pd.to_numeric(df_test_weather['Temperatur_weather'], errors='coerce')





# Drop rows missing predictor values
#df_predicted = df_test_weather.dropna(subset=req_cols).copy()
df_predicted = df_test_weather.copy()



# Predict in next cell
#df_predicted['predicted_Umsatz_umsatz'] = model.predict(df_predicted)

print(df_predicted.shape)
display(df_predicted.head())
display(df_predicted.tail())





Will Predict here

In [ ]:

#predicting with df_test_weather
df_predicted['predicted_Umsatz_umsatz'] = model.predict(df_predicted)

print(df_predicted.shape)
display(df_predicted.head())
display(df_predicted.tail())

## generating data submission csv from df_predicted
Will write a csv file in data named predicted.csv with these columns id,umsatz

In [ ]:
# Prepare output frame and clean missing rows
df_out_predicted = df_predicted[["id", "predicted_Umsatz_umsatz"]].copy()
#rename columns accoding to kaggle submission requirements
df_out_predicted.rename(columns={ "predicted_Umsatz_umsatz": 'umsatz'}, inplace=True)

#print columns name df_out_predicted
print(df_out_predicted.columns)
print(df_out_predicted.shape)

# write to csv
df_out_predicted.to_csv("data/predicted.csv", index=False)
print("Wrote data/predicted.csv with", len(df_out_predicted), "rows.")


